In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
spark.conf.set("sprk.sql.parquet.vorder.enabled","true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, d0efd028-5e97-48c6-926c-4dbda580af1c, 3, Finished, Available)

In [2]:
df = spark.read.format("parquet").load('Tables/Badges')
display(df.limit(10))

StatementMeta(, d0efd028-5e97-48c6-926c-4dbda580af1c, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3b148e3c-56aa-4d95-adbc-17b31f75eef6)

In [6]:
%%sql
SELECT * from Badges
LIMIT 10

StatementMeta(, d0efd028-5e97-48c6-926c-4dbda580af1c, 8, Finished, Available)

<Spark SQL result set with 10 rows and 4 fields>

In [8]:
table_name = 'dim_votes'

dfVotes = spark.read.format("parquet").load('Tables/Votes')
dfVoteTypes = spark.read.format("parquet").load('Tables/VoteTypes')
dfVoteType = dfVotes.join(dfVoteTypes,dfVotes.VoteTypeId == dfVoteTypes.Id, how='Inner')

dfOutputColumns = dfVoteType.select(dfVoteType.PostId, dfVoteType.UserId, dfVoteType.BountyAmount, dfVoteType.Name)
dfOutputColumns.write.mode("overwrite").option("overwriteSchema","true").format("delta").save("Tables/" + table_name)


StatementMeta(, d0efd028-5e97-48c6-926c-4dbda580af1c, 10, Finished, Available)

In [13]:
table_name = 'dim_user'

dfUsers = spark.read.format("parquet").load('Tables/Users')
dfBadges = spark.read.format("parquet").load('Tables/Badges')

dfUserBadge = dfUsers.join(dfBadges, dfUsers.Id == dfBadges.UserId, how='Inner')

dfOutputColumns = dfUserBadge.select(dfUserBadge.DisplayName, dfUserBadge.Age, dfUserBadge.Location, dfUserBadge.Name, dfUserBadge.UserId)
dfOutputColumns.write.mode("overwrite").option("overwriteSchema","true").format("delta").save("Tables/" + table_name)

StatementMeta(, d0efd028-5e97-48c6-926c-4dbda580af1c, 15, Finished, Available)

In [21]:
table_name = 'fact_post_comment'

dfComment = spark.read.format("parquet").load('Tables/Comments')
dfPost = spark.read.format("parquet").load("Tables/Posts")
dfPostComment = dfPost.join(dfComment, dfPost.Id == dfComment.PostId, how='Inner').select(dfPost.OwnerUserId, dfPost.Id, dfPost.AcceptedAnswerId, dfPost.AnswerCount,dfPost.Body, dfPost.ClosedDate,dfPost.CommentCount, dfPost.CreationDate,dfPost.LastActivityDate,dfPost.PostTypeId,dfPost.Score, dfPost.Title, dfPost.Tags, dfComment.CreationDate.alias("CommentDate"), dfComment.Score.alias("CommentScore"), dfComment.Text.alias("CommentText"), dfComment.UserId.alias("CommentUser"))

dfPostComment.write.mode("overwrite").option("overwriteSchema","true").format("delta").save("Tables/" + table_name)


StatementMeta(, d0efd028-5e97-48c6-926c-4dbda580af1c, 23, Finished, Available)